In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# Downsample all platforms to have the same number of data points


In [64]:
df = pd.read_csv('full_dataset.csv')
df.platform.value_counts()

twitter      488167
reddit        72135
instagram     15652
youtube       11304
Name: platform, dtype: int64

In [65]:
df['positivity_normalized'] = (df.positive_words - df.negative_words)/df.word_count
df['intensity_normalized'] = (df.positive_words + df.negative_words)/df.word_count

In [66]:
df = df.dropna()

In [69]:
df_twitter = df.loc[df.platform=='twitter'].sample(11303)
df_reddit = df.loc[df.platform=='reddit'].sample(11303)
df_youtube = df.loc[df.platform=='youtube']
df_instagram = df.loc[df.platform=='instagram'].sample(11303)

In [70]:
df_small = pd.concat([df_twitter, df_reddit, df_youtube, df_instagram])

In [71]:
from sklearn.model_selection import train_test_split

In [83]:
df_train, df_test = train_test_split(df_small.copy(), 
                                     shuffle=True, 
                                     test_size=.2, 
                                     random_state=404,
                                    stratify=df_small.platform)

df_test.platform.value_counts()

youtube      2261
instagram    2261
twitter      2261
reddit       2260
Name: platform, dtype: int64

In [84]:
df_train.to_csv('small_train_dataset.csv')
df_test.to_csv('small_test_dataset.csv')

Positivity is the positive_words and negative_words balance.
Intensity is the sum of pos and neg

,Unnamed: 0,text,platform,positive_words,negative_words,length,word_count,male_words,female_words,afinn,positivity_normalized,intensity_normalized
531834,531834,Grandpa?,reddit,0,0,8,1,0,0,0.0,0.000,0.000000
325553,325553,The sooner we as an Ummah live in accordance t...,twitter,0,0,115,23,0,0,2.0,0.000,0.000000
530669,530669,High school? College?,reddit,0,0,21,3,0,0,0.0,0.000,0.000000
580659,580659,Damn I love this song and this music video,youtube,1,1,42,9,0,0,1.0,0.000,0.222222
576082,576082,He is so messy with the spices lol,youtube,0,1,34,8,2,0,3.0,-0.125,0.125000


# Single feature logistic regression


In [134]:
from sklearn.linear_model import LogisticRegression 
from sklearn.metrics import accuracy_score, confusion_matrix
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import train_test_split

In [135]:
df_train = pd.read_csv('small_train_dataset.csv')
df_test = pd.read_csv('small_test_dataset.csv')

n_splits = 5
kfold = StratifiedKFold(n_splits, shuffle=True, random_state = 404)
features = ['word_count', 'afinn', 'positivity_normalized', 'intensity_normalized', 'male_words', 'female_words']

## Make your array of zeros to hold the accuracies
log_reg_accs = np.zeros((n_splits, len(features)))

In [136]:
## Loop through the cv splits
i = 0
for train_index, test_index in kfold.split(df_train,df_train.platform):
    ## get the training and holdout sets
    df_tt = df_train.iloc[train_index]
    df_ho = df_train.iloc[test_index]
    
    ## loop through your features
    j = 0
    for feature in features:
        ## Define the model
        log_reg = LogisticRegression(penalty='none')
        
        ## fit the model
        log_reg = log_reg.fit(df_tt[feature].values.reshape(-1, 1), df_tt['platform'].values)
        
        ## Make the prediction
        pred = log_reg.predict(df_ho[feature].values.reshape(-1, 1))
        
        ## Record the accuracy on the holdout set
        log_reg_accs[i,j] = accuracy_score(df_ho['platform'].values, pred)
        print(confusion_matrix(df_ho['platform'].values, pred, labels=["instagram", "twitter", "reddit", "youtube"]))
        j = j + 1
    i = i + 1
    
np.mean(log_reg_accs, axis=0)
print(log_reg_accs)

[[  75   80 1342  312]
 [ 180  144  754  731]
 [ 192  183 1101  332]
 [ 130  101 1019  558]]
[[ 558 1183    0   68]
 [ 544 1126    0  139]
 [ 557 1118    0  133]
 [ 623 1057    0  128]]
[[ 490 1294   12   13]
 [ 473 1335    0    1]
 [ 462 1345    0    1]
 [ 480 1292   10   26]]
[[ 546    5 1234   24]
 [ 640    2 1146   21]
 [ 579    1 1193   35]
 [ 615    1 1170   22]]
[[   0    0 1700  109]
 [   0    0 1717   92]
 [   0    0 1737   71]
 [   0    0 1642  166]]
[[  95    0 1714    0]
 [  72    0 1737    0]
 [  43    0 1765    0]
 [  51    0 1757    0]]
[[ 111   61 1289  348]
 [ 168  146  765  730]
 [ 197  192 1093  326]
 [ 170   90 1006  542]]
[[ 608 1127    0   74]
 [ 551 1121    0  137]
 [ 564 1114    0  130]
 [ 606 1086    0  116]]
[[ 568 1221   12    8]
 [ 513 1296    0    0]
 [ 442 1366    0    0]
 [ 478 1295   20   15]]
[[ 575    2 1218   14]
 [ 667    2 1108   32]
 [ 587    0 1191   30]
 [ 609    3 1169   27]]
[[   0    0 1704  105]
 [   0    0 1687  122]
 [   0    0 1727   81]
 

array([0.26638837, 0.25085021, 0.25731986, 0.24686893, 0.26265593,
       0.25748572])

# Findings
Original full dataset:

Here we are fitting logistic regression using word_count, afinn, positivity and instensity individually. 
The confusion matrix has true labels on the rows, prediction on the columns. 
Most model just predict all points as twitter. 
The accuracy was high but the model itself is a trivial one. 
It will not generate well. 

We may need to use balance the dataset by undersample twitter data. 

small dataset: 

accuracy for each featuer decrease to about 26%

# Combined logistic regression

In [114]:
from sklearn import preprocessing

In [127]:
features = ['word_count', 'afinn', 'positivity_normalized', 'intensity_normalized', 'male_words', 'female_words']
df_train = pd.read_csv('small_train_dataset.csv')
df_test = pd.read_csv('small_test_dataset.csv')

n_splits = 5
kfold = StratifiedKFold(n_splits, shuffle=True, random_state = 404)

## Make your array of zeros to hold the accuracies
log_reg_accs = np.zeros((n_splits, 1))

In [129]:
scaler = preprocessing.StandardScaler().fit(df_train[features])
df_train_scaled = scaler.transform(df_train[features])

In [132]:
data = {'word_count': df_train_scaled[:, 0],
         'afinn': df_train_scaled[:, 1],
         'positivity_normalized': df_train_scaled[:, 2],
         'intensity_normalized': df_train_scaled[:, 3],
        'male_words': df_train_scaled[:, 4],
        'female_words': df_train_scaled[:, 5],
         'platform': df_train.platform}
df_train = pd.DataFrame(data)

In [133]:
i = 0
for train_index, test_index in kfold.split(df_train, df_train.platform):
    ## get the training and holdout sets

    df_tt = df_train.iloc[train_index]
    df_ho = df_train.iloc[test_index]
    
    ## Define the model
    log_reg = LogisticRegression(penalty='none')
        
    ## fit the model
    log_reg = log_reg.fit(df_tt[features].values, df_tt.platform.values)
        
    ## Make the prediction
    pred = log_reg.predict(df_ho[features].values)
    ## Record the accuracy on the holdout set
    log_reg_accs[i,0] = accuracy_score(df_ho.platform, pred)
    print(confusion_matrix(df_ho.platform, pred, labels=["instagram", "twitter", "reddit", "youtube"]))

    i = i + 1
    
np.mean(log_reg_accs, axis=0)

[[442 184 997 186]
 [400 638 542 229]
 [441 427 823 117]
 [405 341 704 358]]
[[483 158 952 216]
 [401 616 506 286]
 [433 457 790 128]
 [390 358 708 352]]
[[464 210 961 173]
 [422 601 505 280]
 [453 439 801 116]
 [448 367 654 340]]
[[441 198 930 239]
 [397 622 516 273]
 [436 412 844 117]
 [372 414 702 321]]
[[481 164 967 196]
 [399 621 499 289]
 [494 383 828 104]
 [398 385 670 355]]


array([0.31023821])

# Two class classifier 

Instead of trying to classify them into the 4 bins, train model to try to distinguish one platform from the other three. 

First create one hot encoding for each platform, then add two together to make two-platform bins.


In [160]:
import numpy as np
import pandas as pd

from sklearn.linear_model import LogisticRegression 
from sklearn.metrics import accuracy_score, confusion_matrix
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import train_test_split

In [161]:
df_train = pd.read_csv('small_train_dataset.csv')
df_test = pd.read_csv('small_test_dataset.csv')

In [162]:
# scaled the numerical data
scaler = preprocessing.StandardScaler().fit(df_train[features])
df_train_scaled = scaler.transform(df_train[features])
data = {'word_count': df_train_scaled[:, 0],
         'afinn': df_train_scaled[:, 1],
         'positivity_normalized': df_train_scaled[:, 2],
         'intensity_normalized': df_train_scaled[:, 3],
        'male_words': df_train_scaled[:, 4],
        'female_words': df_train_scaled[:, 5],
         'platform': df_train.platform}
df_train = pd.DataFrame(data)

# create one hot encoding
df_train['dum_twitter'] = 0
df_train.loc[df_train.platform=='twitter', 'dum_twitter'] = 1
df_train['dum_reddit'] = 0
df_train.loc[df_train.platform=='reddit', 'dum_reddit'] = 1
df_train['dum_instagram'] = 0
df_train.loc[df_train.platform=='instagram', 'dum_instagram'] = 1
df_train['dum_youtube'] = 0
df_train.loc[df_train.platform=='youtube', 'dum_youtube'] = 1

df_train['dum_twigram'] = df_train.dum_twitter + df_train.dum_instagram
df_train['dum_twiddit'] = df_train.dum_twitter + df_train.dum_reddit
df_train['dum_twitube'] = df_train.dum_twitter + df_train.dum_youtube

df_train.head()


,word_count,afinn,positivity_normalized,intensity_normalized,male_words,female_words,platform,dum_twitter,dum_reddit,dum_instagram,dum_youtube,dum_twigram,dum_twiddit,dum_twitube
0,-0.282820,-1.284547,-1.414830,0.719725,-0.214425,-0.165903,twitter,1,0,0,0,1,1,1
1,-0.230636,-0.294327,-0.181968,-0.596013,-0.214425,-0.165903,reddit,0,1,0,0,0,1,0
2,0.291204,-1.284547,-0.617096,-0.131635,-0.214425,-0.165903,reddit,0,1,0,0,0,1,0
3,0.030284,-0.294327,-0.181968,-0.596013,-0.214425,-0.165903,instagram,0,0,1,0,1,0,0
4,-0.543739,-0.294327,-0.181968,-0.596013,-0.214425,-0.165903,instagram,0,0,1,0,1,0,0


In [163]:
dums = ['dum_instagram', 'dum_twitter', 'dum_reddit', 'dum_youtube',
        'dum_twigram', 'dum_twiddit', 'dum_twitube']
log_reg_accs = np.zeros([5, len(dums)])
recall = np.zeros([5, len(dums)])
j = 0
for feature in dums:
    i = 0
    for train_index, test_index in kfold.split(df_train, df_train[feature]):
        ## get the training and holdout sets

        df_tt = df_train.iloc[train_index]
        df_ho = df_train.iloc[test_index]

        ## Define the model
        log_reg = LogisticRegression(penalty='none')

        ## fit the model
        log_reg = log_reg.fit(df_tt[features].values, df_tt[feature].values)

        ## Make the prediction
        pred = log_reg.predict(df_ho[features].values)
        ## Record the accuracy on the holdout set
        log_reg_accs[i,j] = accuracy_score(df_ho[feature], pred)
        conf = confusion_matrix(df_ho[feature], pred)
        TN = conf[0,0]
        FP = conf[0,1]
        FN = conf[1,0]
        TP = conf[1,1]
        #print('recall: ', TP/(TP + FN))
        recall[i, j] = TP/(TP + FN)

        i = i + 1
    j += 1
    
#print(log_reg_accs)

In [164]:
print(np.mean(log_reg_accs, axis=0))
print(np.mean(recall,axis=0))

[0.75033869 0.74909454 0.74997927 0.75321409 0.49860373 0.5218005
 0.59697539]
[5.08733373e-03 1.10619469e-04 0.00000000e+00 2.42199157e-02
 2.96671064e-01 6.43903788e-01 4.87115340e-01]


Since twitube vs redgram is the highest accuracy case, run single feature regression to see which one contributed the most. 

In [166]:
df_train.head()

,word_count,afinn,positivity_normalized,intensity_normalized,male_words,female_words,platform,dum_twitter,dum_reddit,dum_instagram,dum_youtube,dum_twigram,dum_twiddit,dum_twitube
0,-0.282820,-1.284547,-1.414830,0.719725,-0.214425,-0.165903,twitter,1,0,0,0,1,1,1
1,-0.230636,-0.294327,-0.181968,-0.596013,-0.214425,-0.165903,reddit,0,1,0,0,0,1,0
2,0.291204,-1.284547,-0.617096,-0.131635,-0.214425,-0.165903,reddit,0,1,0,0,0,1,0
3,0.030284,-0.294327,-0.181968,-0.596013,-0.214425,-0.165903,instagram,0,0,1,0,1,0,0
4,-0.543739,-0.294327,-0.181968,-0.596013,-0.214425,-0.165903,instagram,0,0,1,0,1,0,0


In [170]:
features = ['word_count', 'afinn', 'positivity_normalized', 'intensity_normalized', 'male_words', 'female_words']

log_reg_accs = np.zeros([5, len(features)])
recall = np.zeros([5, len(features)])
j = 0
for feature in features:
    i = 0
    for train_index, test_index in kfold.split(df_train, df_train['dum_twitube']):
        ## get the training and holdout sets

        df_tt = df_train.iloc[train_index]
        df_ho = df_train.iloc[test_index]

        ## Define the model
        log_reg = LogisticRegression(penalty='none')

        ## fit the model
        log_reg = log_reg.fit(df_tt[feature].values.reshape(-1, 1), df_tt['dum_twitube'].values)

        ## Make the prediction
        pred = log_reg.predict(df_ho[feature].values.reshape(-1, 1))
        ## Record the accuracy on the holdout set
        log_reg_accs[i,j] = accuracy_score(df_ho['dum_twitube'], pred)
        conf = confusion_matrix(df_ho['dum_twitube'], pred)
        TN = conf[0,0]
        FP = conf[0,1]
        FN = conf[1,0]
        TP = conf[1,1]
        #print('recall: ', TP/(TP + FN))
        recall[i, j] = TP/(TP + FN)

        i = i + 1
    j += 1
    

In [171]:
print(np.mean(log_reg_accs, axis=0))
print(np.mean(recall,axis=0))

[0.59357481 0.48972886 0.50053906 0.49937842 0.51242776 0.49937784]
[0.46942089 0.60877074 0.72423125 0.46195001 0.07420909 0.22156266]


# Conclusions
Using all features, model was able to tell twitube from redgram with ~60% accuracy. 
